In [ ]:
# Librerias
import pandas as pd
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

In [ ]:
# Lectura y limpieza de datos
datos=[]
ataque=pd.read_csv('data/attacking.csv').drop(['serial','match_played'],axis=1)
intentos=pd.read_csv('data/attempts.csv').drop(['serial','match_played'],axis=1)
defenza=pd.read_csv('data/defending.csv').drop(['serial','match_played'],axis=1)
pases=pd.read_csv('data/distributon.csv').drop(['serial','match_played'],axis=1)
porteros=pd.read_csv('data/goalkeeping.csv').drop(['serial','match_played'],axis=1)
goles=pd.read_csv('data/goals.csv').drop(['serial','match_played'],axis=1)
general=pd.read_csv('data/key_stats.csv').drop(['goals','assists'],axis=1)
datos=[intentos,defenza,pases,porteros,goles,general]
df=ataque
for i in datos:
    df=pd.merge(df,i,on=['player_name','club','position'],how='outer')
df.drop(df[df['player_name']=='Reinildo'].index,inplace=True)
df.drop(df[df['player_name']=='Burak Yilmaz'].index,inplace=True)
df.drop(df[df['player_name']=='Luis Díaz'].index,inplace=True)
df.drop(df[df['player_name']=='Brobbey'].index,inplace=True)
df['g_m']=df['goals']/df['minutes_played'] # Cantidad de goles por minutos jugados
df['m_g']=df['minutes_played']/df['goals'] # Cantidad de minutos para que marque un gol
df['g_p']=df['goals']/df['match_played'] # Cantidad de goles por partidos jugados
df['a_m']=df['assists']/df['minutes_played'] # Cantidad de asistencias por minutos jugados
df['a_p']=df['assists']/df['match_played'] # Cantidad de asistencias por partidos jugados
df.replace([np.inf, -np.inf], np.nan, inplace=True) # eliminar infinitos
df.insert(36,'no_penalties',df['goals']-df['penalties'])
df.fillna(0,inplace=True)
equipos=pd.read_csv('data/club_matches.csv', encoding = 'cp1252').drop(['Unnamed: 0'],axis=1).set_index('club')
equipos=pd.merge(equipos,df.groupby(['club'])['goals'].sum(), on='club')
equipos['g_m']=equipos['goals']/equipos['teams_matches']
df=pd.merge(df,equipos['teams_matches'],on='club')

In [ ]:
# Simular seleccion de jugador
jugador='Benzema'
df_j=df[df['player_name']==jugador].reset_index(drop=True)
df_j.reset_index(drop=True,inplace=True)
df_j

In [ ]:
# Todas las graficas
# Simular seleccion de jugador
jugador='De Bruyne'
df_j=df[df['player_name']==jugador].reset_index(drop=True)
posicion=df_j['position'][0]
df_pos=df[df['position']==posicion]
# Graficar
if df_j['goals'][0]>0: # Grafica Distribucion de goles
    # Datos a graficar
    colores1=cm.get_cmap('Set1', 4).colors
    colores2=cm.get_cmap('Dark2', 2).colors
    colores3=cm.get_cmap('tab10', 2).colors
    x1=df_j.loc[0,'right_foot':'others'][df_j.loc[0,'right_foot':'others']>0]
    x2=df_j.loc[0,'inside_area':'outside_areas'][df_j.loc[0,'inside_area':'outside_areas']>0]
    x3=df_j.loc[0,'penalties':'no_penalties'][df_j.loc[0,'penalties':'no_penalties']>0]
    plt.figure(figsize=(15,10)) # Crear grafica 1
    plt.subplot(111)
    plt.suptitle(f'Distribucion de goles de {jugador}',fontsize=21,fontweight='bold',color='w') # Titulo
    # Crear graficas
    x1.plot.pie(legend=True,colors=colores1,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    x2.plot.pie(legend=True,colors=colores2,radius=0.75,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    x3.plot.pie(legend=True,colors=colores3,radius=0.5,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    # Circulo central
    centre_circle=plt.Circle((0,0),0.25,fc='lightgray')
    fig=plt.gcf()
    fig.gca().add_artist(centre_circle)
    # Texto del circulo
    plt.text(-0.1,0.01,f"Goles: {int(df_j['goals'][0])}",fontdict={'fontweight':'bold','fontsize':12})
    plt.text(-0.15,-0.06,f"Partidos: {int(df_j['match_played'][0])}",fontdict={'fontweight':'bold','fontsize':12})
    plt.axis('equal') # Para que el circulo no tenga forma de elipse
    plt.legend(loc=7,fontsize=12) # Formato de la leyenda
    plt.ylabel('')
    plt.show() # Mostrar
    
# Grafica Comparacion por posicion
if posicion!='Goalkeeper':
    fig,ax=plt.subplots(nrows=3, ncols=2, figsize=(25,13)) # Crear figura
    plt.suptitle(f'Comparacion de {jugador} contra jugadores de su posicion',fontsize=21,fontweight='bold',color='w') # Titulo
    columnas=[['goals','assists'],['g_p','a_p'],['g_m','a_m']] # Columnas a graficar
    # Graficar
    for i,col in zip(range(3),columnas):
        # Graficas izquierda
        df_pos.boxplot([col[0]], ax=ax[i][0],vert=False)
        ax[i][0].plot(df_j[col[0]][0],1, marker="D", color="red",markersize=9)
        ax[i][0].tick_params(labelsize=15,labelcolor='w')
        ax[i][0].set_facecolor('azure')
        # Graficas derecha
        df_pos.boxplot([col[1]], ax=ax[i][1],vert=False)
        ax[i][1].plot(df_j[col[1]][0],1, marker="D", color="red",markersize=9)
        ax[i][1].tick_params(labelsize=15,labelcolor='w')
        ax[i][1].set_facecolor('azure')
    plt.show()
plt.tight_layout()

In [ ]:
# Bigotes por posicion
posicion=df_j['position'][0]
fig,ax=plt.subplots(nrows=3, ncols=2, figsize=(25,10))
plt.title(f'Comparacion de {jugador} contra jugadores de su posicion',fontdict={'fontweight':'bold','color':'w','fontsize':21})
df_pos=df[df['position']==posicion]
columnas=[['goals','assists'],['g_p','a_p'],['g_m','a_m']]
for i,col in zip(range(3),columnas):
    df_pos.boxplot([col[0]], ax=ax[i][0],vert=False)
    ax[i][0].plot(df_j[col[0]][0],1, marker="D", color="red",markersize=8)
    ax[i][0].tick_params(labelsize=15)
    df_pos.boxplot([col[1]], ax=ax[i][1],vert=False)
    ax[i][1].plot(df_j[col[1]][0],1, marker="D", color="red",markersize=8)
    ax[i][1].tick_params(labelsize=15)
plt.show()

In [ ]:
# Distribucion de goles
df_j=df[df['player_name']==jugador].reset_index(drop=True) 
plt.figure(figsize=(14,10))
colores1=cm.get_cmap('Set1', 4).colors
colores2=cm.get_cmap('Dark2', 2).colors
colores3=cm.get_cmap('tab10', 2).colors
x1=df_j.loc[0,'right_foot':'others'][df_j.loc[0,'right_foot':'others']>0]
x2=df_j.loc[0,'inside_area':'outside_areas'][df_j.loc[0,'inside_area':'outside_areas']>0]
x3=df_j.loc[0,'penalties':'no_penalties'][df_j.loc[0,'penalties':'no_penalties']>0]
x1.plot.pie(legend=True,colors=colores1,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
x2.plot.pie(legend=True,colors=colores2,radius=0.75,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
x3.plot.pie(legend=True,colors=colores3,radius=0.5,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
plt.title(f'Distribucion de goles de {jugador}',fontdict={'fontweight':'bold','color':'w','fontsize':21})
centre_circle=plt.Circle((0,0),0.25,fc='lightgray')
fig=plt.gcf()
fig.gca().add_artist(centre_circle)
plt.axis('equal')
plt.legend(loc=7)
plt.text(-0.1,0.01,f"Goles: {int(df_j['goals'][0])}",fontdict={'fontweight':'bold','fontsize':12})
plt.text(-0.13,-0.06,f"Partidos: {int(df_j['match_played'][0])}",fontdict={'fontweight':'bold','fontsize':12})
plt.show()